In [2]:
from nba_api.stats.static import teams, players

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import tensorflow.keras as tfk
from scikeras.wrappers import KerasRegressor

#import scikeras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Masking, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold

fileName = "history_dataset.csv"


random_state = 42 #42

In [ ]:
df = pd.read_csv(fileName)
df = df.dropna()

In [ ]:
df

In [ ]:
from pickle import dump , load
df_mod = df.iloc[:, 5:]

y = df_mod[['y']]
X = df_mod.iloc[:, 1:]

scaler = StandardScaler()
X_std = scaler.fit_transform(X)
X_std = pd.DataFrame(X_std, columns=X.columns)

with open('standardScaler_ready.pkl','wb') as f:
    dump(scaler, f)
    
# y
X_std

In [ ]:
Xtrain_std, Xtest_std, ytrain, ytest = train_test_split(X, y, train_size = 2/3, random_state=random_state)

In [ ]:
input_shape = (Xtrain_std.shape[1],)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=input_shape))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

adam = Adam(learning_rate=0.001)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
callbacks = [EarlyStopping(monitor='loss', patience=10, verbose=1, start_from_epoch=20, restore_best_weights=True)]# start_from_epoch=400, patience=20
history = model.fit(Xtrain_std, ytrain, validation_split=0.25, epochs=32, batch_size=20, callbacks=callbacks, verbose=2)

#history = model.fit(Xtrain_std, ytrain, validation_split=0.25, epochs=32, batch_size=20, verbose=2)

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'validation_loss'], loc='upper left')

plt.show()

In [ ]:
#model.save('modelNBATrained_50.h5')

In [ ]:
trainPred=model.predict(Xtrain_std)
testPred=model.predict(Xtest_std)

trainPred = (trainPred >= 0.5).astype(int)
testPred = (testPred >= 0.5).astype(int)

print(f"Dim Xtest_keras:{len(Xtest_std)}")
trainScore = np.sqrt(mean_squared_error(ytrain, trainPred[:,0]))
testScore = np.sqrt(mean_squared_error(ytest, testPred[:,0]))

print(f"Train score: {trainScore}")
print(f"Test score: {testScore}")

In [ ]:
plt.figure(figsize=(15, 6))
#print(f"len trainPred: {len(trainPred)} \tlen ytrain: {len(ytrain)}")

# Plot dei valori reali
plt.scatter(range(len(ytrain)), ytrain, color='blue', label='Valori reali (ytrain)')

# Plot dei valori predetti
plt.scatter(range(len(trainPred)), trainPred, color='red', label='Valori predetti (trainPred)')

# Impostazioni aggiuntive
plt.xlabel('Numero Pillola')
plt.ylabel('Volume')
plt.title('Confronto TRAIN SET')
plt.legend()

# Imposta il range sull'asse y in base al range dei volumi noto
#plt.ylim(10000, 15000)
#plt.ylim(20000, 100000)
#plt.ylim(15000, 50000)
plt.xlim(1500, 1600)

plt.show()

In [ ]:
plt.figure(figsize=(15, 6))

# Plot dei valori reali
plt.scatter(range(len(ytest)), ytest, color='blue', label='Valori reali (ytest)')

# Plot dei valori predetti
plt.scatter(range(len(testPred)), testPred, color='red', label='Valori predetti (testPred)')

# Impostazioni aggiuntive
plt.xlabel('Numero Pillola')
plt.ylabel('Volume')
plt.title('Confronto TEST SET')
plt.legend()

# Imposta il range sull'asse y in base al range dei volumi noto
#plt.ylim(1000, 90000)
#plt.ylim(10000, 18000)
plt.xlim(0, 40)

# Mostra il grafico
plt.show()

In [ ]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 500)


print(f"ytrain: max: {ytrain.max()} \tmin: {ytrain.min()} \tmean: {ytrain.mean()}")
print(f"ytest: max: {ytest.max()} \tmin: {ytest.min()} \tmean: {ytest.mean()}")

variance_ytest = ytest.values.var()
print(f"\n\nVariance ytest: \t{variance_ytest}")
variance_testPred = np.var(testPred)
print(f"Variance testPred: \t{variance_testPred}")


variance_ytrain = ytrain.values.var()
print(f"\n\nVariance ytrain:\t{variance_ytrain}")
variance_trainPred = np.var(trainPred)
print(f"Variance trainPred:\t{variance_trainPred}")



np.max(np.abs(ytest - testPred.flatten()))
print(f"\n\nytest:")
print(ytest)

massima_differenza = float('-inf')
valore_testPred = None
valore_ytest = None

# Itera entrambi gli array contemporaneamente
ytest_df = pd.DataFrame(ytest)
ytest_df.columns= ["y"]
for val_pred, val_ytest in zip(testPred, ytest_df['y']):
    differenza = abs(val_pred - val_ytest)
    
    # Aggiorna differenza massima e valori corrispondenti se la differenza corrente > massima
    if differenza > massima_differenza:
        massima_differenza = differenza
        valore_testPred = val_pred
        valore_ytest = val_ytest

print(f"\n\nV_testPred: {valore_testPred}, V_ytest: {valore_ytest}, diff maxima: {massima_differenza}")